In [1]:
import cv2
import mediapipe as mp
import time
import numpy as np
import pyautogui
import math
import subprocess
import webbrowser
class HandDetector():
    def __init__(self, mode=False, maxhands=2, detectioncon=0.5, trackcon=0.5):
        self.mode = mode
        self.maxhands = maxhands
        self.detectioncon = detectioncon
        self.trackcon = trackcon
        self.tipid=[4,8,12,16,20]
        self.mphands = mp.solutions.hands
        self.hands = self.mphands.Hands(
            static_image_mode=self.mode,
            max_num_hands=self.maxhands,
            min_detection_confidence=self.detectioncon,
            min_tracking_confidence=self.trackcon
        )
        self.mpdraw = mp.solutions.drawing_utils

    def Findhands(self, img, draw=True):
        imgrgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        self.results = self.hands.process(imgrgb)

        if self.results.multi_hand_landmarks:
            for handlms in self.results.multi_hand_landmarks:
                if draw:
                    self.mpdraw.draw_landmarks(
                        img, handlms, self.mphands.HAND_CONNECTIONS)

        return img
    def Findpos(self,img,handno=0,draw=True):
        self.lmlist=[]
        if self.results.multi_hand_landmarks:
            myhand=self.results.multi_hand_landmarks[handno]
            for id,lm in enumerate(myhand.landmark):
                h,w,c=img.shape
                cx,cy=int(w*lm.x),int(h*lm.y)
#               print(id,cx,cy)
                self.lmlist.append([id,cx,cy])
        return self.lmlist
    
    def Fingersup(self):
        fingers=[]
        if self.lmlist[self.tipid[0]][1]>self.lmlist[self.tipid[0]-1][1]:
            fingers.append(1)
        else:
            fingers.append(0)
        for id in range(1,5):
            if self.lmlist[self.tipid[id]][2]<self.lmlist[self.tipid[id]-2][2]:
                fingers.append(1)
            else:
                fingers.append(0)
        return fingers
    
    def findDistance(self, p1, p2, img, draw=True,r=15, t=3):
        x1, y1 = self.lmlist[p1][1:]
        x2, y2 = self.lmlist[p2][1:]
        cx, cy = (x1 + x2) // 2, (y1 + y2) // 2

        if draw:
            cv2.line(img, (x1, y1), (x2, y2), (255, 0, 255), t)
            cv2.circle(img, (x1, y1), r, (255, 0, 255), cv2.FILLED)
            cv2.circle(img, (x2, y2), r, (255, 0, 255), cv2.FILLED)
            cv2.circle(img, (cx, cy), r, (0, 0, 255), cv2.FILLED)
            length = math.hypot(x2 - x1, y2 - y1)

        return length, img, [x1, y1, x2, y2, cx, cy]
        
        
    

In [3]:
cap.release()

In [2]:

ctime=0
ptime=0
wcam,hcam=640,480 
smoothening = 1
wscr,hscr=pyautogui.size()
framer=100
plocX, plocY = 0, 0
clocX, clocY = 0, 0
cap = cv2.VideoCapture(0)
cap.set(3,wcam)
cap.set(4,hcam)
youtube_open_time=0
l_open_time=0
g_open_time=0
detector = HandDetector(maxhands=1)

while True:
    success, img = cap.read()

    img = detector.Findhands(img)
    lmlist=detector.Findpos(img)
    if len(lmlist)!=0:
        x1,y1=lmlist[8][1:]
        x2,y2=lmlist[12][1:]
        
        fingers=detector.Fingersup()
#         print(fingers)
        cv2.rectangle(img,(50,0),(wcam-framer,hcam-framer-100),(0,255,0),2)
        if fingers[1]==1 and fingers[2]==0:
            x3=np.interp(x1,(50,wcam-framer),(0,wscr))
            y3=np.interp(y1,(0,hcam-framer-100),(0,hscr))
            clocX = plocX + (x3 -plocX) / smoothening
            clocY = plocY + (y3 -plocY) / smoothening
            pyautogui.moveTo(wscr-clocX, clocY)
            cv2.circle(img,(x1,y1),15,(200,2000,0),cv2.FILLED)
        
        if fingers[1] == 1 and fingers[2] == 1:
            # 9. Find distance between fingers
            length, img, lineInfo = detector.findDistance(8, 12, img)
#             print(length)
            # 10. Click mouse if distance short
            if length < 40:
                cv2.circle(img, (lineInfo[4], lineInfo[5]),15, (0, 255, 0), cv2.FILLED)
                pyautogui.click()
                
            
        if fingers[1]==1 and fingers[4]==1:
            if time.time() - g_open_time >= 10:
                # Open another YouTube window only if it's already open and at least 10 seconds have passed
                url = "https://www.geeksforgeeks.org/"
                webbrowser.open(url)
                g_open_time = time.time()
            
        if fingers[0]==1 and fingers[4]==1:
            if time.time() - youtube_open_time >= 10:
                # Open another YouTube window only if it's already open and at least 10 seconds have passed
                url = "https://www.youtube.com"
                webbrowser.open(url)
                youtube_open_time = time.time()
                
        if fingers[2]==1 and fingers[3]==1 and fingers[4]==1:
            if time.time() - l_open_time >= 10:
                # Open another YouTube window only if it's already open and at least 10 seconds have passed
                url = "https://leetcode.com/"
                webbrowser.open(url)
                l_open_time = time.time()

    ctime = time.time()
    fps = 1 / (ctime - ptime)
    ptime = ctime

    cv2.putText(img, str(int(fps)), (10, 70), cv2.FONT_HERSHEY_COMPLEX, 3, (0, 0, 0), 3)

    cv2.imshow("image", img)

        # Exit the loop if the 'q' key is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

    # Release the camera and close the OpenCV window
cap.release()
cv2.destroyAllWindows()

KeyboardInterrupt: 